In [ ]:
import ipcmagic

In [ ]:
%ipcluster start -n 2

In [ ]:
%pxconfig --progress-after -1

In [ ]:
%%px
import glob
import time
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.tfrecord as tfrec
from torchvision import models
from torch.nn.parallel import DistributedDataParallel
from nvidia.dali.pipeline import Pipeline
from nvidia.dali.plugin.pytorch import DALIClassificationIterator, LastBatchPolicy
from pt_distr_env import DistributedEnviron

In [ ]:
%%px
data_dir = '/scratch/snx3000/datasets/imagenet/ILSVRC2012_1k/'

tfrec_files = sorted(glob.glob(f'{data_dir}/train/*'))
index_files = sorted(glob.glob(f'{data_dir}/idx_files/train/*'))

In [ ]:
%%px
distr_env = DistributedEnviron()
dist.init_process_group(backend="nccl")
world_size = dist.get_world_size()
rank = dist.get_rank()
device = 0

In [ ]:
%%px
batch_size = 128

pipe = Pipeline(batch_size=batch_size,
                num_threads=12,
                device_id=device)

with pipe:
    inputs = fn.readers.tfrecord(
        path=tfrec_files,
        index_path=index_files,
        features={
            'image/encoded': tfrec.FixedLenFeature((), tfrec.string, ""),
            'image/class/label': tfrec.FixedLenFeature([1], tfrec.int64,  -1),
        },
        shard_id=rank,
        num_shards=world_size,
        # random_shuffle=True,
        # initial_fill=10000,
        # read_ahead=True,
        # prefetch_queue_depth=20
    )
    jpegs = inputs["image/encoded"]
    labels = inputs["image/class/label"][0] - 1
    images = fn.decoders.image(jpegs, device='mixed', output_type=types.RGB)
    resized = fn.resize(images, device='gpu', size=(224, 224))
    output = fn.crop_mirror_normalize(
        resized,
        dtype=types.FLOAT,
        mean=[0., 0., 0.],
        std=[1., 1., 1.]
    )
    pipe.set_outputs(output.gpu(), labels.gpu())

In [ ]:
%%px
pipe.build()

In [ ]:
%%px
train_loader = DALIClassificationIterator(
    pipe,
    last_batch_padded=False,
    auto_reset=True,
    last_batch_policy=LastBatchPolicy.DROP,
)

In [ ]:
%%px
_model = models.resnet50()
_model.to(device);

ddp_model = DistributedDataParallel(_model, device_ids=[device])

In [ ]:
%%px
optimizer = optim.SGD(ddp_model.parameters(), lr=0.01)

In [ ]:
%%px
def benchmark_step(model, imgs, labels):
    optimizer.zero_grad()
    output = model(imgs)
    loss = F.cross_entropy(output, labels)
    loss.backward()
    optimizer.step()

In [ ]:
%%px
num_epochs = 5
num_iters = 10
imgs_sec = []
for epoch in range(num_epochs):
    t0 = time.time()
    for step, samples in enumerate(train_loader):
        if step > num_iters:
            break
        
        imgs = samples[0]['data']
        labels = samples[0]['label']
        benchmark_step(ddp_model, imgs, labels)

    dt = time.time() - t0
    imgs_sec.append(batch_size * num_iters / dt)

    print(f' * Epoch {epoch:2d}: '
          f'{imgs_sec[epoch]:.2f} images/sec per GPU')

In [ ]:
%ipcluster stop